<a href="https://colab.research.google.com/github/Abhishekconfused/Models/blob/main/Copy_of_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#my project starts here
import numpy as np #array
import pandas as pd #table
import matplotlib.pyplot as plt #plots
import re #substitution
import random #for pusedo random numbers
import nltk #natural language toolkit
from nltk.tokenize import TweetTokenizer
import string
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import collections

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#function for preprocessing. a tweet contains various useless things which are not necessary for our model.I will remove them one by one.
def preprocess(tweet):
  # remove old style retweet text "RT"
  tweett = re.sub(r'^RT[\s]+', '', tweet)

  # remove hyperlinks
  tweettt = re.sub(r'https?:\/\/.*[\r\n]*', '', tweett)

  # remove hashtags
  # only removing the hash # sign from the word
  tweetttt = re.sub(r'#', '', tweettt)
  tweet2= re.sub(r',','',tweetttt)

  #lets tokenize that is break the tweet
  tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                reduce_len=True)
  tweet3= tokenizer.tokenize(tweet2)
    #remove stopwords and panctuations
  import string
  stopwords=nltk.corpus.stopwords.words('english')
  tweet4=[]
  for words in tweet3:
    if words not in stopwords and words not in string.punctuation:
        tweet4.append(words)
  #lets stem
  stemmer=PorterStemmer()
  tweet5=[]
  for words in tweet4:
    stem=stemmer.stem(words)
    tweet5.append(stem)
   #lets lematize
  lemmatizer = WordNetLemmatizer()
  tweet6=[]

  for word in tweet5:
      lem_word = lemmatizer.lemmatize(word)  # stemming word
      tweet6.append(lem_word)  # append to the list

  return tweet6

In [ ]:
#Frequency generating function
#here I will use it to define frequency of a word in posoitive tweert and negative tweet. It will help me to analyse and train the model.
def countfreqs(tweet, ys):
    yslist = np.squeeze(ys).tolist()#the ys which is taken as input is squeezed so that it doesnot be a single element.

    freqs = {}#here I wil pair words with their frequency in a particular emotion
    for y, tweet in zip(yslist, tweet):
        for word in preprocess(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1

    return freqs


In [ ]:

#logical regression model requires sigmoid function so that I can get things between -1 and +1.basically as a number less than 1.
def sigmoid(z):
  h=1/(1+np.exp(-z))
  return h

In [ ]:
def gradientDescent(x, y, theta, alpha, num_iters):

    m = len(x)

    for i in range(0, num_iters):

        # get z, the dot product of x and theta
        z = np.dot(x,theta)

        # get the sigmoid of z
        h = sigmoid(z)

        # calculate the cost function
        J = (-1/m)*(np.dot(y.T,np.log(h)) + np.dot((1-y).T,np.log(1-h)))

        # update the weights theta
        theta = theta - (alpha/m)*np.dot(x.T, h-y)

    J = float(J)
    return J, theta

In [ ]:
def extract_features(tweet, freqs):
  words= preprocess(tweet)
  x = np.zeros((1, 3))#creating an array of size (1,3)
  x[0,0] = 1#I am setting the first term called bias term as 1.
  for word in words:

        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word,1),0)

        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)

  assert(x.shape == (1, 3))
  return x




# MODEL TRAINING.

In [ ]:
p=twitter_samples.strings('positive_tweets.json')
n=twitter_samples.strings('negative_tweets.json')
t=p+n
y = np.append(np.ones((len(p), 1)), np.zeros((len(n), 1)), axis=0)

In [ ]:
#I will train the data using 90:10 ratio.
test_pos = p[4500:]
train_pos = p[:4500]
test_neg = n[4500:]
train_neg = n[:4500]
train_x = train_neg+train_pos
test_x = test_pos + test_neg
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [ ]:
freqs=countfreqs(train_x,train_y)#I am gettint the freqs dictionary using the function .

In [ ]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)
# training labels corresponding to X
Y = train_y
Y = Y.reshape((len(train_y), 1))
# # Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)


In [ ]:
def predict(tweet, freqs, theta):# a fuction which will work on tweets and give me output in a number less than 1.
  x = extract_features(tweet, freqs)#I am storing the features extracted from a tweet in x

  # make the prediction using x and theta
  z = np.dot(x,theta)#I am doing dot of theta fuction defined for the trained data and combining it with the features extracted from the x
  predict = sigmoid(z)#sigmoid function defined above
  return predict





In [ ]:
def test_logistic_regression(test_x, test_y, freqs, theta):
  y_hat = []# storing predictions

  for tweet in test_x:
        y_pred = predict(tweet, freqs, theta)
        if y_pred > 0.5:
            y_hat.append(1) # append 1.0 to the list
        else:
            y_hat.append(0)# append 0 to the list
  y_hat = np.array(y_hat)#converting list in array
  test_y = test_y.reshape(-1)
  accuracy = np.sum((test_y == y_hat).astype(int))/len(test_x)# I will get my predictions in the form of decimal.

  return accuracy


